---
title: "Visualizing Data with Seaborn (Python Track)"
author: 
  name: "Yangyang Li"
  email: yangyang.li@northwestern.edu
date: "2024-09-11"
fig-cap-location: bottom
format: 
  revealjs:
    theme: simple
    slide-number: true
    show-slide-number: all
    chalkboard: false 
    embed-resources: true
    code-block-height: 650px
    transition: slide
    incremental: true
    margin: 0.05
    footer: "Visualizing Data with Seaborn"
execute:
  echo: true
---

## Introduction

- What is Seaborn?
- Why use Seaborn for data visualization?
- Brief overview of the session

## Source Code  


:::: {.columns}

::: {.column width="50%"}
GitHub
![](./qr-code.png)
:::

::: {.column width="50%"}
Colab
:::

::::

## Setting Up the Environment

In [ ]:
!mkdir data && wget -P data/ https://raw.githubusercontent.com/cauliyang/Visualizing-Data-with-Seaborn/main/data/Crimes_One_year_prior_to_present_first_1001.csv

## Setting Up the Environment

In [ ]:
# | output: false
# | code-line-numbers: "|6-18"
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from matplotlib import rcParams

# Set global font properties to Arial
rcParams.update(
    {
        "font.family": "sans-serif",
        "font.sans-serif": "Arial",
        "pdf.fonttype": 42,  # Embed fonts as Type 3 fonts for compatibility
        "ps.fonttype": 42,
        "text.usetex": False,
        "svg.fonttype": "none",
    }
)


def stardize_columns(df):
    df.columns = [" ".join(col.strip().split()) for col in df.columns]
    # Basic data cleaning
    df["DATE OF OCCURRENCE"] = pd.to_datetime(df["DATE OF OCCURRENCE"])


# Load the data
df = pd.read_csv("data/Crimes_One_year_prior_to_present_first_1001.csv")
stardize_columns(df)

## Understanding the Dataset

In [ ]:
# | output: true
# Display basic information about the dataset
print(df.info())

data source: https://data.cityofchicago.org/Public-Safety/Crimes-Map

## Understanding the Dataset

In [ ]:
#| output: asis
from IPython.display import display, HTML

# Convert the first 10 rows of the DataFrame to HTML
df_html = df.head(10).to_html(classes="dataframe", index=False)

# Wrap the table in a div with scrolling
html_content = f"""
<div style="max-height: 400px; overflow: auto;">
    <style>
        .dataframe {{
            font-size: 12px;
            border-collapse: collapse;
            width: 100%;
        }}
        .dataframe th, .dataframe td {{
            border: 1px solid #ddd;
            padding: 8px;
            text-align: left;
        }}
        .dataframe tr:nth-child(even) {{background-color: #f2f2f2;}}
        .dataframe th {{
            background-color: #4CAF50;
            color: white;
            position: sticky;
            top: 0;
        }}
    </style>
    {df_html}
</div>
"""

display(HTML(html_content))

## Introduction to Seaborn Plot Types

- Overview of common Seaborn plot types
- When to use each plot type
- Basic syntax and structure
- Complex plot type

## Categorical Plots: Bar Plot 

In [ ]:
sns.countplot(data=df, y='PRIMARY DESCRIPTION', order=df['PRIMARY DESCRIPTION'].value_counts().index[:10])
plt.title('Top 10 Crime Types')
plt.show()

## Further Exploration 

In [ ]:
from wordcloud import WordCloud
import matplotlib.pyplot as plt


# Combine all secondary descriptions into a single string
text = " ".join(df["SECONDARY DESCRIPTION"].dropna())

# Create and generate a word cloud image
wordcloud = WordCloud(
    width=800, height=400, background_color="white", min_font_size=10
).generate(text)

# Display the generated image
plt.figure(figsize=(12, 8))
plt.imshow(wordcloud, interpolation="bilinear")
plt.axis("off")
plt.show()

## Categorical Plots: Box Plot

In [ ]:
df["DAY_OF_WEEK"] = df["DATE OF OCCURRENCE"].dt.day_name()
plt.figure(figsize=(12, 5))
sns.boxplot(data=df, x="DAY_OF_WEEK", y="DATE OF OCCURRENCE").set_ylabel("Date")
plt.title("Distribution of Crimes by Day of the Week")
plt.show()

## Categorical Plots: Violin Plot

In [ ]:
#| code-line-numbers: "2"
plt.figure(figsize=(12, 5))
sns.violinplot(data=df, x="DAY_OF_WEEK", y="DATE OF OCCURRENCE").set_ylabel("Date")
plt.show()

## Categorical Plots: Box-and-Whisker Plot

In [ ]:
#| code-line-numbers: "2"
plt.figure(figsize=(12, 5))
sns.boxenplot(data=df, x="DAY_OF_WEEK", y="DATE OF OCCURRENCE").set_ylabel("Date")
plt.show()

## Distribution Plots: Histogram

In [ ]:
df["HOUR"] = df["DATE OF OCCURRENCE"].dt.hour
plt.figure(figsize=(12, 5))
sns.histplot(data=df, x="HOUR", bins=24, kde=True)
plt.title("Distribution of Crimes by Hour of the Day")
plt.show()

## Distribution Plots: KDE Plot

In [ ]:
plt.figure(figsize=(12, 6))
sns.kdeplot(data=df, x="HOUR", hue="PRIMARY DESCRIPTION", common_norm=False)
plt.title("Distribution of Different Crime Types by Hour")
plt.show()

## Relational Plots: Scatter Plot

In [ ]:
plt.figure(figsize=(12, 8))
sns.scatterplot(data=df, x="LONGITUDE", y="LATITUDE", hue="PRIMARY DESCRIPTION")
plt.title("Geographical Distribution of Crimes")
plt.show()

## Relational Plots: Line Plot

In [ ]:
crime_counts = df.groupby("DATE OF OCCURRENCE").size().reset_index(name="COUNT")
plt.figure(figsize=(10, 5))
sns.lineplot(data=crime_counts, x="DATE OF OCCURRENCE", y="COUNT")
plt.title("Crime Trends Over Time")
plt.xticks(rotation=45)
plt.show()

## Advanced Customization

In [ ]:
plt.figure(figsize=(14, 6))
sns.set_style("whitegrid")
sns.set_palette("deep")

g = sns.countplot(
    data=df,
    y="PRIMARY DESCRIPTION",
    order=df["PRIMARY DESCRIPTION"].value_counts().index[:10],
)

g.set_title("Top 10 Crime Types", fontsize=20)
g.set_xlabel("Count", fontsize=14)
g.set_ylabel("Crime Type", fontsize=14)

for i, v in enumerate(df["PRIMARY DESCRIPTION"].value_counts()[:10]):
    g.text(v + 3, i, str(v), color="black", va="center")

plt.tight_layout()
plt.show()

## Summary 

![](./function_overview_8_0.png)


## Heatmap

- Useful for visualizing correlation between variables
- Can show patterns and relationships in complex datasets

In [ ]:
# Select numeric columns
numeric_cols = df.select_dtypes(include=[np.number]).columns

# Compute correlation matrix
corr_matrix = df[numeric_cols].corr()

# Create heatmap
plt.figure(figsize=(6, 6))
sns.heatmap(corr_matrix, annot=True, cmap="coolwarm", linewidths=0.5)
plt.xticks(rotation=45, ha="right")
plt.show()  ## Customized Heatmap 

## Customized Heatmap

In [ ]:
#| code-line-numbers: '8'
# Select numeric columns
numeric_cols = df.select_dtypes(include=[np.number]).columns

# Compute correlation matrix
corr_matrix = df[numeric_cols].corr()

# Create a mask for the upper triangle
mask = np.triu(np.ones_like(corr_matrix, dtype=bool))

# Set up the matplotlib figure
plt.figure(figsize=(8, 8))

# Create heatmap with only upper triangle
sns.heatmap(
    corr_matrix,
    mask=mask,
    annot=True,
    cmap="coolwarm",
    vmin=-1,
    vmax=1,
    center=0,
    square=True,
    linewidths=0.5,
    cbar_kws={"shrink": 0.8},
    fmt=".2f",
)
plt.xticks(rotation=45, ha="right")
plt.show()

## Pair Plot 

- Useful for exploring relationships between multiple variables
- Creates a grid of scatter plots for each pair of variables

## Pair Plot {.scrollable}

In [ ]:
# Select relevant columns for the pair plot
cols_to_plot = ["X COORDINATE", "Y COORDINATE", "LATITUDE", "LONGITUDE"]

# Add hour of day
df["HOUR"] = pd.to_datetime(df["DATE OF OCCURRENCE"]).dt.hour

# Create the pair plot
plt.figure(figsize=(5, 5))
pairplot = sns.pairplot(
    df[cols_to_plot + ["HOUR", "PRIMARY DESCRIPTION"]],
    hue="PRIMARY DESCRIPTION",
    palette="viridis",
    plot_kws={"alpha": 0.6},
    diag_kind="kde",
)
plt.tight_layout()
plt.show()

## Regression Plot

- Visualizes the relationship between two variables
- Includes a linear regression line and confidence interval

In [ ]:
#| code-line-numbers: '1'
sns.lmplot(
    data=df,
    x="BEAT",
    y="WARD",
    col="ARREST",
    row="DOMESTIC",
    height=3,
    aspect=2,
    facet_kws=dict(sharex=False, sharey=False),
    scatter_kws={"alpha": 0.5},
)
plt.title("Regression Plot: Latitude vs Longitude of Crime Occurrences")
plt.show()

## Advanced Seaborn: FacetGrid {.scrollable}

- Demonstrates how to create multiple plots in a grid
- Useful for comparing distributions across categories

In [ ]:
# Create a FacetGrid
plt.figure(figsize=(4, 4))
g = sns.FacetGrid(df, col="PRIMARY DESCRIPTION", col_wrap=3, height=4, aspect=1.5)

# Map a histogram to each subplot
g.map(plt.hist, "HOUR", bins=24)

# Customize the plot
g.set_axis_labels("Hour of Day", "Count")
g.set_titles("{col_name}")
g.fig.suptitle("Distribution of Crimes by Hour for Different Crime Types", y=1.02)
g.tight_layout()
plt.show()

## Best Practices and Tips

- Choosing the right plot for your data
- Pay attention to color choices and accessibility
- Avoiding common pitfalls
- Consider the story your visualization is telling


## Q&A 

- Q&A session


## Additional resources  Resours 

- https://seaborn.pydata.org
- https://www.data-to-viz.com/
- https://data.cityofchicago.org/Public-Safety/Crimes-One-year-prior-to-present/x2n5-8w5q/data
- https://quarto.org/docs/presentations/revealjs/